In [1]:
import numpy as np
def EntropyData(dataset):
    n = len(dataset)                        #返回数据集的行数
    dataset=dataset[:,-1]
    count = np.unique(dataset, return_counts=True)[1]
    ent = -np.sum([c/n * np.log2(c/n + 0.00001) for c in count]) # 防止出现log0
    return ent
def maxcount(y):
    y,c = np.unique(y,return_counts=True)
    return y[c==max(c)]
def splitdata(dataset, f, value):
    dataset = dataset[dataset[:, f] == value, :]       
    retDataSet=np.delete(dataset,f,1)
    return retDataSet                                   #返回划分后的数据集
def infoGain(fList, i,dataset):
    baseEntropy = EntropyData(dataset)                 #计算数据集的香农熵
    newEntropy = 0.0                                   #经验条件熵
    for value in fList:                           #计算信息增益
        subDataSet = splitdata(dataset, i, value)           #subDataSet划分后的子集
        prob = len(subDataSet) / float(len(dataset))           #计算子集的概率
        newEntropy += prob * EntropyData(subDataSet)        #根据公式计算经验条件熵
    infoGain = baseEntropy - newEntropy                        #信息增益
    return infoGain
def choose(dataset):
    numFeatures = len(dataset[0]) - 1                     #特征数量  
    bestInfoGain = 0.0                                    #信息增益
    bestFeature = -1                                      #最优特征的索引值
    for i in range(numFeatures):                          #遍历所有特征
        #获取dataSet的第i个所有特征
        featList = [example[i] for example in dataset]
        uniqueVals = set(featList)                         #创建set集合{},元素不可重复
        iGain = infoGain(uniqueVals,i,dataset)                        #信息增益
        print("第%d个特征的增益为%.3f" % (i, iGain))             #打印每个特征的信息增益
        if (iGain > bestInfoGain):                              #计算信息增益
            bestInfoGain = iGain                                #更新信息增益，找到最大的信息增益
            bestFeature = i                                        #记录信息增益最大的特征的索引值
    return bestFeature                                             #返回信息增益最大的特征的索引值

def createTree(dataSet, labels, featLabels):
    classList = [example[-1] for example in dataSet]               #取分类标签(是否放贷:yes or no)
    if classList.count(classList[0]) == len(classList):            #如果类别完全相同则停止继续划分
        return classList[0]
    if len(dataSet[0]) == 1:                                       #遍历完所有特征时返回出现次数最多的类标签
        return maxcount(classList)
    bestFeat = choose(dataSet)                   #选择最优特征
    bestFeatLabel = labels[bestFeat]                               #最优特征的标签
    featLabels.append(bestFeatLabel)
    myTree = {bestFeatLabel:{}}                                    #根据最优特征的标签生成树
    del(labels[bestFeat])                                          #删除已经使用特征标签
    featValues = [example[bestFeat] for example in dataSet]        #得到训练集中所有最优特征的属性值
    uniqueVals = set(featValues)                                   #去掉重复的属性值
    for value in uniqueVals:
        subLabels=labels[:]
        #递归调用函数createTree(),遍历特征，创建决策树。
        myTree[bestFeatLabel][value] = createTree(splitdata(dataSet, bestFeat, value), subLabels, featLabels)
    return myTree

dataSet = [[0, 0, 0, 0, 'no'],         #数据集
           [0, 0, 0, 1, 'no'],
           [0, 1, 0, 1, 'yes'],
           [0, 1, 1, 0, 'yes'],
           [0, 0, 0, 0, 'no'],
           [1, 0, 0, 0, 'no'],
           [1, 0, 0, 1, 'no'],
           [1, 1, 1, 1, 'yes'],               
           [1, 0, 1, 2, 'yes'],
           [1, 0, 1, 2, 'yes'],
           [2, 0, 1, 2, 'yes'],
           [2, 0, 1, 1, 'yes'],
           [2, 1, 0, 1, 'yes'],
           [2, 1, 0, 2, 'yes'],
           [2, 0, 0, 0, 'no']]
dataSet=np.array(dataSet)
labels = ['年龄', '有工作', '有自己的房子', '信贷情况']        #分类属性
featLabels = []
myTree = createTree(dataSet, labels, featLabels)
print(myTree)




第0个特征的增益为0.083
第1个特征的增益为0.324
第2个特征的增益为0.420
第3个特征的增益为0.363
第0个特征的增益为0.252
第1个特征的增益为0.918
第2个特征的增益为0.474
{'有自己的房子': {'0': {'有工作': {'0': 'no', '1': 'yes'}}, '1': 'yes'}}


In [2]:
import numpy as np
def EntropyData(dataset):
    n = len(dataset)                        #返回数据集的行数
    dataset=dataset[:,-1]
    count = np.unique(dataset, return_counts=True)[1]
    ent = -np.sum([c/n * np.log2(c/n + 0.00001) for c in count]) # 防止出现log0
    return ent
def maxcount(y):
    y,c = np.unique(y,return_counts=True)
    return y[c==max(c)]
def splitdata(dataset, f, value):
    dataset = dataset[dataset[:, f] <= value, :]       
    retDataSet=np.delete(dataset,f,1)
    return retDataSet                                   #返回划分后的数据集
def infoGain(fList, i,dataset):
    baseEntropy = EntropyData(dataset)                 #计算数据集的香农熵
    newEntropy = 0.0                                   #经验条件熵
    for value in fList:                           #计算信息增益
        subDataSet = splitdata(dataset, i, value)           #subDataSet划分后的子集
        prob = len(subDataSet) / float(len(dataset))           #计算子集的概率
        newEntropy += prob * EntropyData(subDataSet)        #根据公式计算经验条件熵
    infoGain = baseEntropy - newEntropy                        #信息增益
    return infoGain
def infoGain_rate(fList,i,dataset):
    H = EntropyData(dataset)
    IG = infoGain(fList,i,dataset)
    return IG/H
def choose_C4_5(dataset):
    numFeatures = len(dataset[0]) - 1                     #特征数量  
    bestInfoGainRate = 0.0                                    #信息增益
    bestFeature = -1                                      #最优特征的索引值
    for i in range(numFeatures):                          #遍历所有特征
        #获取dataSet的第i个所有特征
        featList = [example[i] for example in dataset]
        uniqueVals = set(featList)                         #创建set集合{},元素不可重复
        iGain = infoGain_rate(uniqueVals,i,dataset)                        #信息增益
        print("第%d个特征的增益率为%.3f" % (i, iGain))             #打印每个特征的信息增益
        if (iGain > bestInfoGainRate):                              #计算信息增益
            bestInfoGainRate = iGain                                #更新信息增益，找到最大的信息增益
            bestFeature = i                                        #记录信息增益最大的特征的索引值
    return bestFeature                                             #返回信息增益最大的特征的索引值

def createTreeC4_5(dataSet, labels, featLabels):
    classList = [example[-1] for example in dataSet]               #取分类标签(是否放贷:yes or no)
    if classList.count(classList[0]) == len(classList):            #如果类别完全相同则停止继续划分
        return classList[0]
    if len(dataSet[0]) == 1:                                       #遍历完所有特征时返回出现次数最多的类标签
        return maxcount(classList)
    bestFeat = choose_C4_5(dataSet)                   #选择最优特征
    bestFeatLabel = labels[bestFeat]                               #最优特征的标签
    featLabels.append(bestFeatLabel)
    myTree = {bestFeatLabel:{}}                                    #根据最优特征的标签生成树
    del(labels[bestFeat])                                          #删除已经使用特征标签
    featValues = [example[bestFeat] for example in dataSet]        #得到训练集中所有最优特征的属性值
    uniqueVals = set(featValues)                                   #去掉重复的属性值
    for value in uniqueVals:
        subLabels=labels[:]
        #递归调用函数createTree(),遍历特征，创建决策树。
        myTree[bestFeatLabel][value] = createTreeC4_5(splitdata(dataSet, bestFeat, value), subLabels, featLabels)
    return myTree

dataSet = [[0, 0, 0, 0, 'no'],         #数据集
           [0, 0, 0, 1, 'no'],
           [0, 1, 0, 1, 'yes'],
           [0, 1, 1, 0, 'yes'],
           [0, 0, 0, 0, 'no'],
           [1, 0, 0, 0, 'no'],
           [1, 0, 0, 1, 'no'],
           [1, 1, 1, 1, 'yes'],               
           [1, 0, 1, 2, 'yes'],
           [1, 0, 1, 2, 'yes'],
           [2, 0, 1, 2, 'yes'],
           [2, 0, 1, 1, 'yes'],
           [2, 1, 0, 1, 'yes'],
           [2, 1, 0, 2, 'yes'],
           [2, 0, 0, 0, 'no']]
dataSet=np.array(dataSet)
labels = ['年龄', '有工作', '有自己的房子', '信贷情况']        #分类属性
featLabels = []
myTree = createTree(dataSet, labels, featLabels)
print(myTree)




第0个特征的增益为-0.990
第1个特征的增益为-0.647
第2个特征的增益为-0.551
第3个特征的增益为-0.970
第0个特征的增益为-1.268
第1个特征的增益为0.000
第2个特征的增益为0.000
第0个特征的增益为-1.338
第1个特征的增益为-1.047
第2个特征的增益为-0.835
第0个特征的增益为-1.200
第1个特征的增益为0.000
第0个特征的增益为-1.200
第0个特征的增益为-0.888
第1个特征的增益为-0.647
第0个特征的增益为-0.794
第0个特征的增益为-0.911
第0个特征的增益为-0.918
第1个特征的增益为-0.377
第0个特征的增益为-0.893
第0个特征的增益为-1.135
{'信贷情况': {'no': {'有自己的房子': {'0': '0', '1': '0'}}, 'yes': {'有自己的房子': {'0': {'有工作': {'0': '0', '1': {'年龄': {'0': array(['0'], dtype='<U1'), '1': array(['0'], dtype='<U1')}}}}, '2': {'有工作': {'0': {'年龄': {'0': array(['0'], dtype='<U1'), '1': array(['0'], dtype='<U1')}}, '1': {'年龄': {'0': array(['1'], dtype='<U1'), '1': array(['0', '1', '2'], dtype='<U1')}}}}, '1': {'有工作': {'0': {'年龄': {'0': array(['0'], dtype='<U1'), '1': array(['0'], dtype='<U1')}}, '1': {'年龄': {'0': array(['0'], dtype='<U1'), '1': array(['0'], dtype='<U1')}}}}}}}}
